In [1]:
#!pip install selenium pandas beautifulsoup4

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup Chrome browser in headless mode
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

# Get user input
keyword = input("Enter product search keyword: ").replace(" ", "+")
base_url = f"https://www.flipkart.com/search?q={keyword}"

# Empty lists to store data
product_names = []
prices = []
ratings = []

# Scrape first 3 pages
for page in range(1, 4):
    url = f"{base_url}&page={page}"
    print(f"Scraping Page {page}...")
    driver.get(url)
    time.sleep(4)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Select correct product containers for laptops & electronics
    product_blocks = soup.find_all("div", class_="_1AtVbE col-12-12")

    for block in product_blocks:
        title = block.find("div", class_="_4rR01T")
        if not title:
            continue  # skip non-product blocks

        price = block.find("div", class_="_30jeq3 _1_WHN1")
        rating = block.find("div", class_="_3LWZlK")

        if title and price:
            product_names.append(title.text.strip())
            prices.append(price.text.strip().replace("₹", "").replace(",", ""))
            ratings.append(rating.text.strip() if rating else "N/A")

# Close browser
driver.quit()

# Create DataFrame
df = pd.DataFrame({
    "Product Name": product_names,
    "Price (INR)": prices,
    "Rating": ratings
})

# Print and Save
print("\n✅ Scraped Data:\n")
print(df)

df.to_csv("flipkart_products.csv", index=False)
print("\n✅ Data saved to 'flipkart_products.csv'")


Enter product search keyword:  laptops


Scraping Page 1...
Scraping Page 2...
Scraping Page 3...

✅ Scraped Data:

Empty DataFrame
Columns: [Product Name, Price (INR), Rating]
Index: []

✅ Data saved to 'flipkart_products.csv'
